In [1]:
import scipy.misc
import numpy as np
import matplotlib 
%matplotlib notebook
matplotlib.use('TkAgg')
#%matplotlib inline
import matplotlib.pyplot as plt
import warnings
import cv2
from pylab import *
import matplotlib.lines as mlines

In [2]:
image = cv2.imread('HW2/img.jpg')
#marking top and bottom points of sign board whose height is known
plt.imshow(image)
known_points =plt.ginput(2)
known_top = known_points[0]
known_bottom = known_points[1]
plt.close()
#plt.plot([known_top[0], known_bottom[0]], [known_top[1], known_bottom[1]], marker = 'o')

In [3]:
#marking top and bottom points of an object whose height is unknown
plt.imshow(image)
unknown_points =plt.ginput(2)
unknown_top = unknown_points[0]
unknown_bottom = unknown_points[1]
plt.close()
#plt.plot([unknown_top[0], unknown_bottom[0]], [unknown_top[1], unknown_bottom[1]], marker = 'o')

In [4]:
#function to calculate the vanishing points
def get_intersection_points():
    plt.imshow(image)
    pl_points =plt.ginput(4)
    plt.close()
    print(pl_points)
    slope=[]
    intercept=[]
    for i in range(0,len(pl_points),2):
        slope.append((pl_points[i+1][1]-pl_points[i][1])/(pl_points[i+1][0]-pl_points[i][0]))
        intercept.append(pl_points[i][1]-(pl_points[i][0]*slope[-1]))
    if(slope[0] == slope[1]):
        return None
    else:
        x = (intercept[1] - intercept[0]) /(slope[0] - slope[1])
        y = slope[0]*x + intercept[0]
        return((x,y))

In [5]:
#finding vanishing points
vp1 = get_intersection_points()
vp2 = get_intersection_points()
if(vp1 != None and vp2!= None):
    print(vp1,vp2)
else:
    print("parallel lines.")


[(938.4202608878604, 928.7051335998312), (647.5723753418897, 977.1797811908264), (943.8063328424155, 1074.129076372817), (652.9584472964448, 1106.445508100147)]
[(1676.3121186618978, 1084.9012202819267), (1848.666421207658, 1079.5151483273721), (1676.3121186618978, 1375.749105827898), (1843.2803492531034, 1381.135177782453)]
(-1689.9828529349666, 1366.772319236969) (-2903.3879204113196, 1228.0168465029542)


In [6]:
#finding vanishing line equation
vl_m = (vp1[1] - vp2[1])/(vp1[0] - vp2[0])
vl_c = vp1[1] - vl_m*vp1[0]
print('equation of vanishing line : y = ' + str(vl_m) + 'x + ' + str(vl_c))

equation of vanishing line : y = 0.11435214542378602x + 1560.0254841994931


In [ ]:
plt_img = cv2.imread('HW2/img.jpg')
plt.imshow(plt_img)
plt.plot([int(vp1[0]),int(vp2[0])],[int(vp1[1]),int(vp2[1])])
plt.show()

In [7]:
#calculating eq. of line drawn from known object bottom to vanishing line
bp_m = (known_bottom[1]-unknown_bottom[1])/(known_bottom[0]-unknown_bottom[0])
bp_c = known_bottom[1] - bp_m * known_bottom[0]
print('equation of projection on vanishing line : y = ' + str(bp_m) + 'x + ' + str(bp_c))

equation of projection on vanishing line : y = 0.1470588235294118x + 2046.7658166359327


In [8]:
#calculating intersection of vanishing line and previous projected line
proj_pt_x = (bp_c-vl_c)/(vl_m-bp_m)
proj_pt_y = vl_m*proj_pt_x + vl_c
print(proj_pt_x, proj_pt_y)

-14881.98620674708 -141.76156670922683


In [9]:
#calculating line from previous calculated point and object top
rl_m = (unknown_top[1] - proj_pt_y) / (unknown_top[0] - proj_pt_x)
rl_c = unknown_top[1] - unknown_top[0] * rl_m
print('equation of reference line : y = ' + str(rl_m) + 'x + ' + str(rl_c))

equation of reference line : y = 0.13574031763242553x + 1878.3239679959975


In [10]:
#calculating equation of object with known height
ref_m = (known_bottom[1]-known_top[1])/(known_bottom[0]-known_top[0])
ref_c = known_top[1] - ref_m * known_top[0]
print('equation of line along reference object : y = ' + str(ref_m) + 'x + ' + str(ref_c))

equation of line along reference object : y = -24.99999999999983x + 28218.70881548478


In [11]:
#calculating refernce point on known object
ref_x = (ref_c - rl_c)/(rl_m-ref_m)
ref_y = rl_m*ref_x + rl_c
ref = (ref_x,ref_y) 
print(ref_x,ref_y)

1047.925563943366 2020.5697169008085


In [12]:
#applying cross ration to calculate height of object
cross_ratio = np.linalg.norm(np.array(ref)-np.array(known_bottom))/np.linalg.norm(np.array(known_top) - np.array(known_bottom))
cross_ratio *= 1.65
print(cross_ratio)

2.1964790604550792


In [13]:
temp_m = (known_top[1] - known_bottom[1])/(known_top[0] - known_bottom[0])
temp_c = known_top[1] - temp_m*known_top[0]

temp_x = (temp_c - vl_c)/(vl_m - temp_m)
temp_y = temp_x*temp_m + temp_c
temp = (temp_x,temp_y)

#applying cross ration to find the height of camera
cam_cross_ratio = np.linalg.norm(np.array(temp)-np.array(known_bottom))/np.linalg.norm(np.array(known_top) - np.array(known_bottom))
cam_cross_ratio *= 1.65
print(cam_cross_ratio)

6.35249166778941
